In [104]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, OrdinalEncoder, FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split


In [55]:
accounts = pd.read_csv('ravenstack_accounts.csv')
cevents = pd.read_csv('ravenstack_churn_events.csv')
fusage = pd.read_csv('ravenstack_feature_usage.csv')
subscription = pd.read_csv('ravenstack_subscriptions.csv')
support = pd.read_csv('ravenstack_support_tickets.csv')

In [56]:
accounts.sample(10)

,account_id,account_name,industry,country,signup_date,referral_source,plan_tier,seats,is_trial,churn_flag
297,A-a865bd,Company_297,Cybersecurity,US,2023-09-19,organic,Pro,38,False,False
223,A-7c6b6b,Company_223,FinTech,US,2024-11-30,other,Basic,6,False,False
56,A-1b707d,Company_56,Cybersecurity,US,2023-07-14,organic,Enterprise,1,False,False
445,A-dec005,Company_445,HealthTech,US,2023-10-13,partner,Pro,21,False,False
325,A-cbd1a3,Company_325,DevTools,US,2024-11-25,other,Basic,6,False,True
96,A-8b25f2,Company_96,Cybersecurity,US,2024-02-17,ads,Pro,6,False,True
252,A-dbc825,Company_252,EdTech,FR,2024-01-07,partner,Enterprise,19,False,False
152,A-439b2f,Company_152,DevTools,US,2024-02-21,ads,Enterprise,1,False,True
107,A-039727,Company_107,EdTech,AU,2024-11-02,organic,Pro,25,False,False
294,A-342303,Company_294,EdTech,US,2024-03-16,other,Pro,1,True,False


In [57]:
accounts.shape

(500, 10)

In [58]:
accounts.columns

Index(['account_id', 'account_name', 'industry', 'country', 'signup_date',
       'referral_source', 'plan_tier', 'seats', 'is_trial', 'churn_flag'],
      dtype='object')

In [59]:
accounts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   account_id       500 non-null    object
 1   account_name     500 non-null    object
 2   industry         500 non-null    object
 3   country          500 non-null    object
 4   signup_date      500 non-null    object
 5   referral_source  500 non-null    object
 6   plan_tier        500 non-null    object
 7   seats            500 non-null    int64 
 8   is_trial         500 non-null    bool  
 9   churn_flag       500 non-null    bool  
dtypes: bool(2), int64(1), object(7)
memory usage: 32.4+ KB


In [60]:
accounts.describe()

,seats
count,500.000000
mean,20.560000
std,21.044718
min,1.000000
25%,5.000000
50%,15.000000
75%,28.000000
max,163.000000


In [61]:
accounts.duplicated().sum()

np.int64(0)

In [62]:
accounts['signup_date'] = pd.to_datetime(accounts['signup_date'])

In [63]:
accounts['industry'].unique()

array(['EdTech', 'FinTech', 'DevTools', 'HealthTech', 'Cybersecurity'],
      dtype=object)

In [64]:
accounts['country'].unique()

array(['US', 'IN', 'UK', 'CA', 'DE', 'AU', 'FR'], dtype=object)

In [65]:
accounts['referral_source'].unique()

array(['partner', 'other', 'organic', 'event', 'ads'], dtype=object)

In [66]:
accounts['plan_tier'].unique()

array(['Basic', 'Enterprise', 'Pro'], dtype=object)

In [67]:
pd.crosstab(accounts['industry'], accounts['churn_flag'], normalize = 'index')

churn_flag,False,True
industry,,
Cybersecurity,0.840000,0.160000
DevTools,0.690265,0.309735
EdTech,0.835443,0.164557
FinTech,0.776786,0.223214
HealthTech,0.781250,0.218750


In [68]:
pd.crosstab(accounts['country'], accounts['churn_flag'], normalize = 'index')

churn_flag,False,True
country,,
AU,0.875000,0.125000
CA,0.826087,0.173913
DE,0.680000,0.320000
FR,0.772727,0.227273
IN,0.795918,0.204082
UK,0.810345,0.189655
US,0.766323,0.233677


In [69]:
reference_date = accounts["signup_date"].max()
accounts["account_age_days"] = (reference_date - accounts["signup_date"]).dt.days

In [70]:
accounts["tenure_bucket"] = pd.cut(
    accounts["account_age_days"],
    bins=[0,30,90,180,365,10000],
    labels=["0-30","30-90","90-180","180-365","365+"]
)

In [71]:
accounts["signup_month"] = accounts["signup_date"].dt.month
accounts["signup_year"] = accounts["signup_date"].dt.year

In [72]:
accounts.sample(2)

,account_id,account_name,industry,country,signup_date,referral_source,plan_tier,seats,is_trial,churn_flag,account_age_days,tenure_bucket,signup_month,signup_year
14,A-956988,Company_14,EdTech,UK,2024-11-30,ads,Enterprise,1,False,False,31,30-90,11,2024
381,A-0a62f5,Company_381,FinTech,UK,2024-09-11,partner,Pro,16,False,False,111,90-180,9,2024


In [73]:
pd.crosstab(accounts['tenure_bucket'], accounts['churn_flag'], normalize = 'index')

churn_flag,False,True
tenure_bucket,,
0-30,0.937500,0.062500
30-90,0.825397,0.174603
90-180,0.814286,0.185714
180-365,0.780488,0.219512
365+,0.744493,0.255507


In [74]:
pd.crosstab(accounts['signup_month'], accounts['churn_flag'], normalize = 'index')

churn_flag,False,True
signup_month,,
1,0.787879,0.212121
2,0.645161,0.354839
3,0.808511,0.191489
4,0.783784,0.216216
5,0.770833,0.229167
6,0.735294,0.264706
7,0.850000,0.150000
8,0.864865,0.135135
9,0.708333,0.291667


In [75]:
pd.crosstab(accounts['signup_year'], accounts['churn_flag'], normalize = 'index')

churn_flag,False,True
signup_year,,
2023,0.744493,0.255507
2024,0.809524,0.190476


In [76]:
pd.crosstab(accounts['referral_source'], accounts['churn_flag'], normalize = 'index')

churn_flag,False,True
referral_source,,
ads,0.765306,0.234694
event,0.697917,0.302083
organic,0.824561,0.175439
other,0.757282,0.242718
partner,0.853933,0.146067


In [77]:
pd.crosstab(accounts['plan_tier'], accounts['churn_flag'], normalize = 'index')

churn_flag,False,True
plan_tier,,
Basic,0.779762,0.220238
Enterprise,0.779221,0.220779
Pro,0.780899,0.219101


In [78]:
pd.crosstab(accounts['is_trial'], accounts['churn_flag'], normalize = 'index')

churn_flag,False,True
is_trial,,
False,0.789082,0.210918
True,0.742268,0.257732


***Feature Engineering on Accounts df***


In [79]:
accounts.sample(2)

,account_id,account_name,industry,country,signup_date,referral_source,plan_tier,seats,is_trial,churn_flag,account_age_days,tenure_bucket,signup_month,signup_year
118,A-42e5e1,Company_118,DevTools,US,2024-07-09,ads,Pro,1,False,True,175,90-180,7,2024
431,A-c16b88,Company_431,HealthTech,US,2023-12-02,event,Enterprise,23,False,False,395,365+,12,2023


In [80]:
X = accounts.drop(columns = ['churn_flag'])
y = accounts['churn_flag']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)

In [123]:
preprocessor = ColumnTransformer(
    [
        ('ohe_indus_cou_rsource',
         OneHotEncoder(sparse_output=False, drop='first'),
         [2,3,5]),

        ('oe_plan_tier',
         OrdinalEncoder(categories=[['Basic','Pro','Enterprise']]),
         [6])
    ],
    remainder='passthrough'
)

In [124]:
pipe = Pipeline([('preprocessor', preprocessor)])

In [125]:
pipe.fit_transform(X_train)
pipe.transform(X_test)

array([[0.0, 0.0, 0.0, ..., '365+', 1, 2023],
       [0.0, 0.0, 0.0, ..., '0-30', 12, 2024],
       [0.0, 0.0, 0.0, ..., '30-90', 11, 2024],
       ...,
       [0.0, 0.0, 1.0, ..., '365+', 12, 2023],
       [0.0, 0.0, 1.0, ..., '365+', 6, 2023],
       [0.0, 0.0, 1.0, ..., '365+', 9, 2023]], dtype=object)

In [81]:
ohe = OneHotEncoder(sparse_output = False, drop = 'first')
X_train_ohe = ohe.fit_transform(X_train[['industry', 'country', 'referral_source']])
X_test_ohe = ohe.transform(X_test[['industry', 'country', 'referral_source']])


In [103]:
pd.DataFrame(X_train_ohe, columns = ohe.get_feature_names_out())

,industry_DevTools,industry_EdTech,industry_FinTech,industry_HealthTech,country_CA,country_DE,country_FR,country_IN,country_UK,country_US,referral_source_event,referral_source_organic,referral_source_other,referral_source_partner
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
396,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
397,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
398,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


In [83]:
cevents.sample(2)

,churn_event_id,account_id,churn_date,reason_code,refund_amount_usd,preceding_upgrade_flag,preceding_downgrade_flag,is_reactivation,feedback_text
159,C-fd2fe4,A-78f02b,2024-12-09,budget,0.0,False,False,False,too expensive
447,C-0c829f,A-c7ffc2,2023-04-15,budget,0.0,False,False,False,too expensive


In [84]:
cevents.shape

(600, 9)

In [85]:
cevents.columns

Index(['churn_event_id', 'account_id', 'churn_date', 'reason_code',
       'refund_amount_usd', 'preceding_upgrade_flag',
       'preceding_downgrade_flag', 'is_reactivation', 'feedback_text'],
      dtype='object')

In [86]:
cevents.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   churn_event_id            600 non-null    object 
 1   account_id                600 non-null    object 
 2   churn_date                600 non-null    object 
 3   reason_code               600 non-null    object 
 4   refund_amount_usd         600 non-null    float64
 5   preceding_upgrade_flag    600 non-null    bool   
 6   preceding_downgrade_flag  600 non-null    bool   
 7   is_reactivation           600 non-null    bool   
 8   feedback_text             452 non-null    object 
dtypes: bool(3), float64(1), object(5)
memory usage: 30.0+ KB


In [87]:
fusage.sample(2)

,usage_id,subscription_id,usage_date,feature_name,usage_count,usage_duration_secs,error_count,is_beta_feature
13636,U-032738,S-240a09,2023-01-07,feature_21,10,4940,3,False
18424,U-50f7ed,S-8c10c1,2024-03-27,feature_1,12,60,0,False


In [88]:
fusage.shape

(25000, 8)

In [89]:
fusage.columns

Index(['usage_id', 'subscription_id', 'usage_date', 'feature_name',
       'usage_count', 'usage_duration_secs', 'error_count', 'is_beta_feature'],
      dtype='object')

In [90]:
fusage.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   usage_id             25000 non-null  object
 1   subscription_id      25000 non-null  object
 2   usage_date           25000 non-null  object
 3   feature_name         25000 non-null  object
 4   usage_count          25000 non-null  int64 
 5   usage_duration_secs  25000 non-null  int64 
 6   error_count          25000 non-null  int64 
 7   is_beta_feature      25000 non-null  bool  
dtypes: bool(1), int64(3), object(4)
memory usage: 1.4+ MB


In [91]:
fusage.describe()

,usage_count,usage_duration_secs,error_count
count,25000.000000,25000.000000,25000.000000
mean,10.021000,3042.202880,0.564280
std,3.143729,2056.544615,1.012595
min,0.000000,0.000000,0.000000
25%,8.000000,1350.000000,0.000000
50%,10.000000,2760.000000,0.000000
75%,12.000000,4400.000000,1.000000
max,26.000000,12696.000000,8.000000


In [92]:
subscription.sample(2)

,subscription_id,account_id,start_date,end_date,plan_tier,seats,mrr_amount,arr_amount,is_trial,upgrade_flag,downgrade_flag,churn_flag,billing_frequency,auto_renew_flag
3264,S-3196e2,A-ee42db,2024-01-15,NaN,Basic,13,0,0,True,False,False,False,annual,True
848,S-5eb846,A-2789e3,2024-12-10,NaN,Pro,11,539,6468,False,False,False,False,monthly,False


In [93]:
subscription.shape

(5000, 14)

In [94]:
subscription.columns

Index(['subscription_id', 'account_id', 'start_date', 'end_date', 'plan_tier',
       'seats', 'mrr_amount', 'arr_amount', 'is_trial', 'upgrade_flag',
       'downgrade_flag', 'churn_flag', 'billing_frequency', 'auto_renew_flag'],
      dtype='object')

In [95]:
subscription.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   subscription_id    5000 non-null   object
 1   account_id         5000 non-null   object
 2   start_date         5000 non-null   object
 3   end_date           486 non-null    object
 4   plan_tier          5000 non-null   object
 5   seats              5000 non-null   int64 
 6   mrr_amount         5000 non-null   int64 
 7   arr_amount         5000 non-null   int64 
 8   is_trial           5000 non-null   bool  
 9   upgrade_flag       5000 non-null   bool  
 10  downgrade_flag     5000 non-null   bool  
 11  churn_flag         5000 non-null   bool  
 12  billing_frequency  5000 non-null   object
 13  auto_renew_flag    5000 non-null   bool  
dtypes: bool(5), int64(3), object(6)
memory usage: 376.1+ KB


In [96]:
subscription.describe()

,seats,mrr_amount,arr_amount
count,5000.000000,5000.000000,5000.000000
mean,29.852000,2267.749400,27212.992800
std,23.089771,3421.375348,41056.504178
min,1.000000,0.000000,0.000000
25%,14.000000,285.000000,3420.000000
50%,24.000000,931.000000,11172.000000
75%,40.000000,2786.000000,33432.000000
max,189.000000,33830.000000,405960.000000


In [97]:
support.sample(2)

,ticket_id,account_id,submitted_at,closed_at,resolution_time_hours,priority,first_response_time_minutes,satisfaction_score,escalation_flag
492,T-adcdc9,A-adc1f3,2023-06-12,2023-06-14 23:00:00,71.0,urgent,25,4.0,False
210,T-c722ef,A-ced9aa,2023-09-17,2023-09-17 18:00:00,18.0,low,69,NaN,False


In [98]:
support.shape

(2000, 9)

In [99]:
support.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 9 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   ticket_id                    2000 non-null   object 
 1   account_id                   2000 non-null   object 
 2   submitted_at                 2000 non-null   object 
 3   closed_at                    2000 non-null   object 
 4   resolution_time_hours        2000 non-null   float64
 5   priority                     2000 non-null   object 
 6   first_response_time_minutes  2000 non-null   int64  
 7   satisfaction_score           1175 non-null   float64
 8   escalation_flag              2000 non-null   bool   
dtypes: bool(1), float64(2), int64(1), object(5)
memory usage: 127.1+ KB


In [100]:
support.describe()

,resolution_time_hours,first_response_time_minutes,satisfaction_score
count,2000.000000,2000.000000,1175.000000
mean,35.861000,88.480000,3.981277
std,21.138427,51.531877,0.809646
min,1.000000,1.000000,3.000000
25%,17.000000,43.000000,3.000000
50%,35.000000,88.000000,4.000000
75%,54.000000,131.000000,5.000000
max,72.000000,180.000000,5.000000
